In [88]:
import pandas as pd 
import numpy as np
import re
import os


folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [89]:
df = pd.read_csv('vehicles_df.csv')
df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66408 entries, 0 to 66407
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          66408 non-null  int64  
 1   Public Vehicle Number               66408 non-null  int64  
 2   Status                              66408 non-null  object 
 3   Vehicle Make                        58740 non-null  object 
 4   Vehicle Model                       58556 non-null  object 
 5   Vehicle Model Year                  58640 non-null  float64
 6   Vehicle Color                       58464 non-null  object 
 7   Vehicle Fuel Source                 66408 non-null  object 
 8   Wheelchair Accessible               66408 non-null  object 
 9   Company Name                        66408 non-null  object 
 10  Address                             59264 non-null  object 
 11  City                                59264

In [91]:
(df['Wheelchair Accessible']=='Y').sum()

2496

In [92]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [93]:
(df['Vehicle Model Year'].apply(lambda x: isinstance(x,float))).sum()

66408

In [94]:
df['Vehicle Model Year'].unique()

array([2014., 2010., 2008., 2015., 2012., 2016., 2011., 2017., 2009.,
       2013., 2024., 2005., 2001., 2002., 2000., 1993., 2007., 1998.,
       2006., 1997., 1988., 1990., 2004., 1999., 1986., 1992., 2021.,
       2020., 2023., 2018., 2019., 2022.,   nan, 2003., 2025., 1996.,
       1984., 1994., 1981., 1995., 1991., 1989., 1983., 1987., 1985.,
          7., 1980., 1196.,    0.])

In [95]:

values_replace = [7.,0.]
df['Vehicle Model Year']=df['Vehicle Model Year'].replace(values_replace,np.nan)


In [96]:
df['Vehicle Model Year']=df['Vehicle Model Year'].fillna(0)

In [97]:
df['Vehicle Model Year']=df['Vehicle Model Year'].astype(int)

In [98]:
df.tail()

,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,Address,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID
66403,42,ACTIVE,PRECISION,6 SEATER,2020,FUSCIA,Pedal,N,JBL PEDICAB MANAGEMENT LLC,1627 N. TRIPP AVE.,CHICAGO,IL,60639.0,NaN,NaN,42Pedicab
66404,52,ACTIVE,SCHWINN,NONE,2014,GREEN,Pedal,N,BM PEDICAB INC.,4450 N. KEDZIE AVE.,CHICAGO,IL,60625.0,NaN,NaN,52Pedicab
66405,70,VIOLATION,MAINSTREET,CLASSIC,0,RED/BLACK,Pedal,N,EMPIRE PEDICAB LLC,318 W. ADAMS ST.,CHICAGO,IL,60606.0,NaN,NaN,70Pedicab
66406,57,VIOLATION,MAINSTREET,BOARDWALK,2011,RED,Pedal,N,CARGILL C KELLY II,2428 W. 24TH PL.,CHICAGO,IL,60608.0,NaN,NaN,57Pedicab
66407,117,ACTIVE,SCHWINN,NONE,2014,GREEN,Pedal,N,BM PEDICAB INC.,4450 N. KEDZIE AVE.,CHICAGO,IL,60625.0,NaN,NaN,117Pedicab


In [99]:
df['Vehicle Make'].unique()

array(['CHEVROLET', 'MERCEDES', 'VAN HOOL', 'VPG', 'DODGE', 'TOYOTA',
       'FORD', 'CADILLAC', 'FREIGHLINE', 'CHAMPION', 'THOMAS',
       'FREIGHTLINER', 'MCI', nan, 'ALEXANDER DENNIS LTD', 'GILLIG',
       'INTL', 'LEYLAND', 'ALEXDER DENNIS LTD', 'IC', 'DOUBLE SHUFLE',
       'ISUZU', 'SPEC-CONSTED', 'TESLA', 'KIA', 'TUK TUK', 'MAINSTREET',
       'LUCID', 'POLARIS', 'BMW', 'PRECISION', 'CHRYSLER', 'MERCURY',
       'LINCOLN', 'GMC', 'ALEXANDER', 'VOLVO', 'HONDA', 'HYUNDAI',
       'NISSAN', 'AUDI', 'LEXUS', 'INFINITI', 'JEEP', 'MOVITRON', 'ORION',
       'ANKAI', 'MASERATI', 'MAZDA', 'CHARLESTON', 'NEOPLAN', 'GENISIS',
       'OSHKOSH', 'BLUEBIRD', 'VOLKSWAGEN', 'BOYERTOWN', 'MITSUBISHI',
       'MOBILITY VENTURE', 'LAND ROVER', 'TURTLE TOP', 'SUBARU', 'ACURA',
       'SCION', 'Dodge', 'Ford', 'PONTIAC', 'BMX', 'BARTH', 'JAGUAR',
       'CINDERELLA', 'TROYER', 'KIMBALL', 'YODER', 'MARTIN', 'SCHWARTZ',
       'MILLER', 'ANDERSON', 'SCHROCK', 'VICTORIA', 'STUDEBAKER',
       'MANMADE

In [100]:
df['ZIP Code']=df['ZIP Code'].fillna(0)

In [101]:
df['ZIP Code']=df['ZIP Code'].astype(int)

In [102]:
df.drop_duplicates(subset=['Public Vehicle Number'],inplace=True)

In [103]:
df.drop(columns=['Address'],inplace=True)

In [104]:
df.drop(columns=['Public Vehicle Number'],inplace=True)

In [105]:
public_column = df['Record ID']

In [106]:
public_column

0        12009Charter Sightseeing
1        12248Charter Sightseeing
2        13527Charter Sightseeing
4        13528Charter Sightseeing
5        12025Charter Sightseeing
                   ...           
16409                     998Taxi
16410                    3810Taxi
16411                    4690Taxi
16412                    2831Taxi
16413                    1509Taxi
Name: Record ID, Length: 14473, dtype: object

In [107]:
df['Vehicle Type'] = df['Record ID'].str.extract(r'([a-zA-Z\s]+)$')

In [108]:
df['Taxi Affiliation']=df['Taxi Affiliation'].fillna('none')

In [109]:
df['Taxi Medallion License Management ']=df['Taxi Medallion License Management '].fillna('none')

In [110]:
df['State'].isna().sum()

1688

In [111]:
df.tail()

,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
16409,VIOLATION,TOYOTA,CAMRY,2012,WHITE,Hybrid,N,AGOMONY CAB INC,CHICAGO,IL,60659,DIS-AFFILIATED,Owner Manager,998Taxi,Taxi
16410,ACTIVE,TOYOTA,RAV4,2017,GRAY/WHITE,Hybrid,N,FENKIL ASMERA TAXI COMPANY,CHICAGO,IL,60613,Choice Taxi Association Inc,Owner Manager,3810Taxi,Taxi
16411,VIOLATION,TOYOTA,CAMRY,2014,WHITE,Hybrid,N,MURIELLE CADICHON,CHICAGO,IL,60643,DIS-AFFILIATED,Owner-Operator,4690Taxi,Taxi
16412,ACTIVE,TOYOTA,PRIUS,2012,WHITE,Hybrid,N,G S TRANS INC,CHICAGO,IL,60659,City Service Taxi Association,Owner Manager,2831Taxi,Taxi
16413,ACTIVE,TOYOTA,SIENNA,2023,WHITE,Hybrid,Y,J A SHALOM LLC,CHICAGO,IL,60612,Independent1,Owner-Operator,1509Taxi,Taxi


In [112]:
Q1 = df['Vehicle Model Year'].quantile(0.25)
Q3 = df['Vehicle Model Year'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Vehicle Model Year'] < lower_bound) | (df['Vehicle Model Year'] > upper_bound)]

print("Outliers:\n", outliers)

Outliers:
          Status Vehicle Make Vehicle Model  Vehicle Model Year Vehicle Color  \
134    INACTIVE     CHAMPION       TROLLEY                1993        ORANGE   
146    INACTIVE      LEYLAND      OLYMPIAN                1988      BURGUNDY   
151    INACTIVE     CHAMPION       TROLLEY                1990         GREEN   
194    INACTIVE      LEYLAND      OLYMPIAN                1988      BURGUNDY   
198    INACTIVE        ISUZU      CARRYALL                1986        MAROON   
...         ...          ...           ...                 ...           ...   
14353  INACTIVE          NaN           NaN                   0         WHITE   
15238  INACTIVE          NaN           NaN                   0         WHITE   
15700  INACTIVE          NaN           NaN                   0         WHITE   
16090  INACTIVE          NaN           NaN                   0         WHITE   
16396  INACTIVE          NaN           NaN                   0         WHITE   

      Vehicle Fuel Source Wh

In [113]:
df['Vehicle Model Year'].value_counts()

Vehicle Model Year
0       1820
2023    1395
2014    1302
2013    1151
2012    1040
2022     993
2019     740
2024     708
2011     705
2015     700
2016     669
2020     556
2021     545
2017     487
2018     453
2010     267
2008     167
2006     154
2007     109
2009     107
2005      64
2001      57
2025      50
2004      43
2002      35
2003      29
1999      27
1997      22
2000      20
1998      13
1996       7
1991       5
1988       5
1992       4
1995       4
1993       3
1984       3
1989       2
1994       2
1986       2
1990       2
1981       1
1983       1
1987       1
1985       1
1980       1
1196       1
Name: count, dtype: int64

In [114]:
df['State'].unique()

array(['IL', nan], dtype=object)

In [115]:
df=df.dropna(subset=['State'])

In [116]:
df['State'].isna().sum()

0

In [117]:
df.shape

(12785, 15)

In [118]:
df['Vehicle Make'].isna().sum()

113

In [119]:
df2=df.sample(50)

In [120]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'CHIICAGO', 'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON',
       'CHICAGO RIDGE', 'BARTLETT', 'EVERGREEN PARK', 'ARLINGTON HEIGHTS',
       'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK', 'CHGO', 'PLAINFIELD',
       'DEERFIELD', 'OAKBROOK', 'NORTHBROOK', 'BENSENVILLE',
       'ELK GROVE VILLAGE', 'JOHNSBURG', 'NORRIDGE', 'DESPLAINES',
       'DES PLAINES', 'ITASCA', 'CHCAGO', 'GURNEE', 'METTAWA'],
      dtype=object)

In [121]:
valid_cities = [
    'CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE', 'FRANKFORT', 'ELGIN', 'SHOREWOOD', 
    'TINLEY PARK', 'LYNWOOD', 'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON', 'CHICAGO RIDGE', 
    'BARTLETT', 'EVERGREEN PARK', 'ARLINGTON HEIGHTS', 'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK', 
    'PLAINFIELD', 'DEERFIELD', 'OAK BROOK', 'NORTHBROOK', 'BENSENVILLE', 'ELK GROVE VILLAGE', 'JOHNSBURG', 
    'NORRIDGE', 'DES PLAINES', 'ITASCA', 'GURNEE', 'METTAWA'
]

In [122]:
def correct_city_name(city):
    
    city_corrections = {
        'CHIICAGO': 'CHICAGO',
        'CHGO': 'CHICAGO',
        'OAKBROOK': 'OAK BROOK',
        'DESPLAINES': 'DES PLAINES',
        'CHCAGO': 'CHICAGO'
    }
  
    return city_corrections.get(city, city)

In [123]:
df['City'] = df['City'].apply(correct_city_name)

In [124]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON', 'CHICAGO RIDGE',
       'BARTLETT', 'EVERGREEN PARK', 'ARLINGTON HEIGHTS', 'BROOKFIELD',
       'EARLVILLE', 'SCHILLER PARK', 'PLAINFIELD', 'DEERFIELD',
       'OAK BROOK', 'NORTHBROOK', 'BENSENVILLE', 'ELK GROVE VILLAGE',
       'JOHNSBURG', 'NORRIDGE', 'DES PLAINES', 'ITASCA', 'GURNEE',
       'METTAWA'], dtype=object)

In [125]:
df = df[df['City'].isin(valid_cities)]

In [126]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON', 'CHICAGO RIDGE',
       'BARTLETT', 'EVERGREEN PARK', 'ARLINGTON HEIGHTS', 'BROOKFIELD',
       'EARLVILLE', 'SCHILLER PARK', 'PLAINFIELD', 'DEERFIELD',
       'OAK BROOK', 'NORTHBROOK', 'BENSENVILLE', 'ELK GROVE VILLAGE',
       'JOHNSBURG', 'NORRIDGE', 'DES PLAINES', 'ITASCA', 'GURNEE',
       'METTAWA'], dtype=object)

In [127]:
city_coordinates = {
    'CHICAGO': (41.8781, -87.6298),
    'ELMHURST': (41.8991, -87.9403),
    'ORLAND PARK': (41.6295, -87.8534),
    'ST. CHARLES': (41.9140, -88.3052),
    'SKOKIE': (42.0324, -87.7411),
    'FRANKFORT': (41.3102, -87.8506),
    'ELGIN': (42.0354, -88.2812),
    'SHOREWOOD': (41.7799, -88.2281),
    'TINLEY PARK': (41.5742, -87.7820),
    'LYNWOOD': (41.5742, -87.6232),
    'LOVES PARK': (42.3019, -89.0543),
    'BARRINGTON': (42.1531, -88.1382),
    'NEW LENOX': (41.5130, -87.9812),
    'DOLTON': (41.6190, -87.5936),
    'CHICAGO RIDGE': (41.7150, -87.7820),
    'BARTLETT': (41.9951, -88.1851),
    'EVERGREEN PARK': (41.7107, -87.7893),
    'ARLINGTON HEIGHTS': (42.0884, -87.9806),
    'BROOKFIELD': (41.8314, -87.7846),
    'EARLVILLE': (41.6274, -88.6947),
    'SCHILLER PARK': (41.9781, -87.8182),
    'PLAINFIELD': (41.6303, -88.2039),
    'DEERFIELD': (42.1546, -87.8472),
    'OAK BROOK': (41.8503, -87.9511),
    'NORTHBROOK': (42.1211, -87.8294),
    'BENSENVILLE': (41.9783, -87.9401),
    'ELK GROVE VILLAGE': (42.0353, -87.9732),
    'JOHNSBURG': (42.3191, -88.2733),
    'NORRIDGE': (41.9810, -87.7947),
    'DES PLAINES': (42.0396, -87.8704),
    'ITASCA': (41.9534, -88.0169),
    'GURNEE': (42.3620, -87.8837),
    'METTAWA': (42.2573, -87.9116)
}

In [128]:
df['Latitude'] = df['City'].map(lambda city: city_coordinates.get(city, (None, None))[0])
df['Longitude'] = df['City'].map(lambda city: city_coordinates.get(city, (None, None))[1])

In [129]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON', 'CHICAGO RIDGE',
       'BARTLETT', 'EVERGREEN PARK', 'ARLINGTON HEIGHTS', 'BROOKFIELD',
       'EARLVILLE', 'SCHILLER PARK', 'PLAINFIELD', 'DEERFIELD',
       'OAK BROOK', 'NORTHBROOK', 'BENSENVILLE', 'ELK GROVE VILLAGE',
       'JOHNSBURG', 'NORRIDGE', 'DES PLAINES', 'ITASCA', 'GURNEE',
       'METTAWA'], dtype=object)

In [130]:
print(df.sample(5))

          Status Vehicle Make Vehicle Model  Vehicle Model Year Vehicle Color  \
13795     ACTIVE       TOYOTA         CAMRY                2017        YELLOW   
11528     ACTIVE       TOYOTA          RAV4                2016         WHITE   
11574  VIOLATION       TOYOTA         PRIUS                2014         WHITE   
3298      ACTIVE         FORD          E450                2017         WHITE   
11800     ACTIVE       TOYOTA          RAV4                2024        SILVER   

      Vehicle Fuel Source Wheelchair Accessible           Company Name  \
13795              Hybrid                     N            YZ TAXI INC   
11528              Hybrid                     N     ENO MARY TRANS INC   
11574              Hybrid                     N          JSP CAB CORP.   
3298             Gasoline                     N  ANN-ETHE SERVICES LLC   
11800              Hybrid                     N          KPOOKEKE  INC   

          City State  ZIP Code               Taxi Affiliation  \
137

In [131]:
df.head()

,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type,Latitude,Longitude
0,RESERVED,CHEVROLET,EXPRESS,2014,BLACK,Bio-Diesel,N,CHICAGO PRIVATE TOURS LLC,CHICAGO,IL,60653,none,none,12009Charter Sightseeing,Charter Sightseeing,41.8781,-87.6298
1,INACTIVE,MERCEDES,SPRINTER,2010,SILVER,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640,none,none,12248Charter Sightseeing,Charter Sightseeing,41.8781,-87.6298
2,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609,none,none,13527Charter Sightseeing,Charter Sightseeing,41.8781,-87.6298
4,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609,none,none,13528Charter Sightseeing,Charter Sightseeing,41.8781,-87.6298
5,INACTIVE,MERCEDES,SPRINTER,2015,BLACK,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640,none,none,12025Charter Sightseeing,Charter Sightseeing,41.8781,-87.6298


In [137]:
df['Taxi Medallion License Management '].value_counts()

Taxi Medallion License Management 
none                                      6426
Owner Manager                             3417
Owner-Operator                            1939
Medallion Leasing and Management, Inc.     340
Star North Management LLC                  180
Cab Management Group, Inc.                 139
Sun Financial Services, Inc.               129
Top Cab Corp.                              122
Chicago Taxicab Management Inc.             89
Medallion Management Corp.                   2
Taxi Experts, Inc.                           2
Name: count, dtype: int64

In [142]:

city_meds_count = df[df['City'] == 'CHICAGO'].groupby('Taxi Medallion License Management ').size()


In [143]:
city_meds_count

Taxi Medallion License Management 
Cab Management Group, Inc.                 139
Chicago Taxicab Management Inc.             89
Medallion Leasing and Management, Inc.     340
Medallion Management Corp.                   2
Owner Manager                             3416
Owner-Operator                            1937
Star North Management LLC                  180
Sun Financial Services, Inc.               129
Taxi Experts, Inc.                           2
Top Cab Corp.                              122
none                                      5693
dtype: int64